In [1]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR + 'code/RNN/lstm/rnn_lstm_model.ipynb'

In [2]:
%run {RUN_DIR}

In [ ]:
class RnnExtModel(RnnLstmModel):
    pass

In [ ]:
def rnn_ext_alloc_seqwrap_layer(self, input_shape, hconfig):
    self.pms = []
    prev_shape = input_shape[1:]
    if not isinstance(hconfig[1], dict): hconfig.insert(1, {})
        
    for hconfig in hconfig[2:]:
        pm, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        self.pms.append(pm)
        
    if isinstance(prev_shape, int): prev_shape = [prev_shape]
    output_shape = [input_shape[0]] + list(prev_shape)
    
    return self.pms, output_shape

RnnExtModel.alloc_seqwrap_layer = rnn_ext_alloc_seqwrap_layer

In [ ]:
def rnn_ext_init_parameters(self, hconfigs):
    self.hconfigs = hconfigs
    self.pm_hiddens = []

    prev_shape = self.dataset.input_shape

    for hconfig in hconfigs:
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        
        if len(pm_hidden) > 1 : 
            pm_hidden = sum(pm_hidden,[])
            self.pm_hiddens.append(pm_hidden)
            self.pm_hiddens.append([nn.Flatten(60, -1)])
            self.pm_hiddens.append([nn.Linear(144, 16)])
            re=reshape_net()
            self.pm_hiddens.append([re])
        else:
            self.pm_hiddens.append(pm_hidden)
        
    output_cnt = int(np.prod(self.dataset.output_shape))
    
    self.pm_hiddens.append([nn.Flatten()])
    
    self.seqout = False

    if len(hconfigs) > 0 and get_layer_type(hconfigs[-1]) in ['rnn', 'lstm']:
        if get_conf_param(hconfigs[-1], 'outseq', True):
            self.seqout = True
            prev_shape = prev_shape[1:]
            output_cnt = int(np.prod(self.dataset.output_shape[1:]))

    self.pm_output, _ = self.alloc_layer_param(prev_shape, output_cnt)
    
    self.layers=sum(self.pm_hiddens,[])
RnnExtModel.init_parameters = rnn_ext_init_parameters

In [ ]:
class Net(nn.Module):
    def __init__(self, layers):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(*layers)
        
    def forward(self, x):
        print(x.shape,'shape')
        x=x.reshape(-1, 90, 120, 3)
        print(x.shape,'shape')
        x=x.transpose(1,-1)
        print(x.shape,'shape')
        x=x.transpose(2,-1)
        print(x.shape,'shape')
        out = self.layer1(x)
#         out.reshape()
        return out

In [ ]:
# nn.Module.forward(x=x.reshape(10,-1,16))

In [ ]:
class reshape_net(nn.Module):
    def __init__(self):
        super(reshape_net,self).__init__()
#         self.reshape_layer = nn.Sequential()
        
    def forward(self,x):
        print(x.shape)
        x=x.reshape(10,-1,16)
        return x

In [ ]:
# def get_train_loader(self):
#     train_loader=self.dataset.dataloader(self.batch_size, self.num_workers)
#     return train_loader

# RnnExtModel.get_train_loader = get_train_loader

In [ ]:
# train_loader=self.dataset.dataloader(self.dataset.tr_xs, self.dataset.tr_ys, batch_size, num_workers)